In [75]:
from ioMicro import *
def get_tzxy_fast(im1,im2,sz=6):
    ncols,Sz,Sx,Sy = im1.shape
    im1_ = im1[-1,((Sz-sz)//2):((Sz+sz)//2)]
    im2_ = im2[-1,((Sz-sz)//2):((Sz+sz)//2)]
    txyz,txyzs = get_txyz(im1_,im2_,sz_norm=25,sz = 400,nelems=5)
    return txyz,txyzs
import os,sys,glob,numpy as np
def get_dic_fls(analysis_fls):
    dic_fls = {} 
    for fl in analysis_fls:
        ifov = get_ifov(fl)
        if ifov not in dic_fls: dic_fls[ifov] = []
        if '_Q' in os.path.basename(fl):
            dic_fls[ifov]+=[fl]
    return dic_fls

class post_analysis():
    def __init__(self,analysis_fls = r'Y:\Glass_MERFISH\CGBB_ChrX_01_31_2023_Analysis\*_C*_Xh_RNAs.pkl',force=False):
        self.save_folder = self.analysis_folder =os.path.dirname(analysis_fls)
        save_Afls = self.save_folder+os.sep+'analysis_fls.npy'
        if os.path.exists(save_Afls) and not force:
            self.analysis_fls = np.load(save_Afls)
        else:
            self.analysis_fls = np.array(glob.glob(analysis_fls))
            np.save(save_Afls,self.analysis_fls)
        self.dic_fls = get_dic_fls(self.analysis_fls)
        
    def get_Xh(ifov=0,iH=0,icol=0):
        self.iH = iH
        self.icol = icol
        self.ifov=ifov
        
        self.fit_fls = self.dic_fls[self.ifov]
        self.fit_fl = self.fit_fls[self.iH]
        self.Xhs,self.dic_drift = pickle.load(open(fl,'rb'))
        self.Xh = Xhs[self.icol]
    def load_segm(self,force=False):
        analysis_folder = self.save_folder
        self.fov= fov = get_fov(self.fl)
        segm_folder = analysis_folder+os.sep+'Segmentation'
        dapi_fls_fl = segm_folder+os.sep+'dapi_fls_fl.npy'
        if os.path.exists(dapi_fls_fl) or force:
            dapi_fls = glob.glob(segm_folder+os.sep+'*.npz')
            np.save(dapi_fls_fl,dapi_fls)
        else:
            dapi_fls = np.load(dapi_fls_fl)
        self.dapi_fls = dapi_fls
        self.fl_dapi = fl_dapi = [fl for fl in self.dapi_fls if fov in fl and self.set_ in get_H_fld(fl)][0]
        #save_fl  = segm_folder+os.sep+os.path.basename(fl_dapi).split('.')[0]+'--'+os.path.basename(os.path.dirname(fl_dapi))+'--dapi_segm.npz'
        self.im_segm,self.shape = np.load(fl_dapi)['segm'],np.load(fl_dapi)['shape']
        self.im_segmf = resize(self.im_segm,self.shape)
    def get_Xh_Q(self,ifov=0,Q = 'Q1'):
        if True:#try:
            self.ifov=ifov
            self.fit_fls = self.dic_fls[self.ifov]
            fls = self.fit_fls
            iQ = int(Q[1:])
            iQs = [list(eval(fl.split('Q')[-1].split('_')[0])) for fl in fls]
            self.iH,self.icol = [[iH_,iQ_.index(iQ)] for iH_,iQ_ in enumerate(iQs) if iQ in iQ_][0]
            self.fl = self.fit_fls[self.iH]
            if getattr(self,"current_fl","")!=self.fl:
                self.current_fl = self.fl
                self.Xhs,self.dic_drift = pickle.load(open(self.fl,'rb'))
            self.Xh = self.Xhs[self.icol]
            self.Q=Q
            return 1
        else:#except:
            return -1
def plot_Xh(self,viewer=None,color='white',dic_th={},rescale=True):
    Xh = self.Xh.copy()
    
    cor = Xh[:,-2]
    h = Xh[:,-1]
    th_br = dic_th.get(self.icol,0)
    Xh = Xh[h>th_br]
    
    h = Xh[:,-1]
    #import napari
    #napari.view_points()
    #bad = cor>0.25
    if rescale:
        min_,max_ = np.sort(h)[np.array([50,-10])%len(h)]
        h_n = np.clip((h-min_)/(max_-min_),0,1)
        size=20*h_n
    else:
        size=10
    import napari
    if viewer is None:
        viewer = napari.Viewer()
    viewer.add_points(Xh[:,:3],size=size,name = self.Q,face_color=color)#,text=h.astype(int).astype(str))
    return viewer

def add_png_image(self,viewer=None,return_im=False):
    import cv2,napari
    self.im_png = im_png = cv2.imread(self.fl.replace('--_Xh_RNAs.pkl','_signal-col'+str(self.icol)+'.png'))
    im_ = im_png[...,0]
    xs = np.where(np.diff(np.percentile(im_,5,axis=1)>254)!=0)[0]+1
    xm,xM = np.min(xs),np.max(xs)
    ys = np.where(np.diff(np.percentile(im_,5,axis=0)>254)!=0)[0]+1
    ym,yM = np.min(ys),np.max(ys)
    im_=im_[xm:xM,ym:yM]
    new_shape = [1]+list(np.max(self.Xh[:,:3],axis=0).astype(int)[1:])
    im__ = resize(im_[np.newaxis],new_shape)
    if return_im:
        return im__
    if viewer is None: viewer = napari.Viewer()
    viewer.add_image(im__)
def get_fov(fl): return os.path.basename(fl).split('--')[0].split('.')[0]
def load_segm(self,force=False,vol_th = 1000,dil=5,stitch_th=0.5):
    self.prev_fov = getattr(self,"prev_fov","")
    self.fov= fov = get_fov(self.fl) 
    if (self.prev_fov!=self.fov) or force or (not hasattr(self,"im_segm")):
        analysis_folder = self.save_folder
        self.prev_fov = fov
        segm_folder = analysis_folder+os.sep+'Segmentation'
        dapi_fls_fl = segm_folder+os.sep+'dapi_fls_fl.npy'
        if not os.path.exists(dapi_fls_fl) or force:
            dapi_fls = glob.glob(segm_folder+os.sep+'*.npz')
            np.save(dapi_fls_fl,dapi_fls)
        else:
            dapi_fls = np.load(dapi_fls_fl)
        self.dapi_fls = dapi_fls
        self.fl_dapi = fl_dapi = [fl for fl in self.dapi_fls if self.fov in fl and self.set_ in get_H_fld(fl)][0]
        #save_fl  = segm_folder+os.sep+os.path.basename(fl_dapi).split('.')[0]+'--'+os.path.basename(os.path.dirname(fl_dapi))+'--dapi_segm.npz'
        self.im_segm,self.shape = np.load(fl_dapi)['segm'],np.load(fl_dapi)['shape']
        self.im_segm = stitch3D(self.im_segm,niter=5,th_int=stitch_th)
        #self.im_segmf = resize(self.im_segm,self.shape)
        import scipy.ndimage as nd
        vols = nd.sum(self.im_segm>-1,self.im_segm,np.unique(self.im_segm))

        bad_cells = np.where(vols<vol_th)[0]
        bad = np.in1d(self.im_segm,bad_cells).reshape(self.im_segm.shape)
        self.im_segm[bad]=0

        self.im_segm_bin = self.im_segm>0
        self.im_segm_bin_dil = nd.binary_dilation(self.im_segm_bin,iterations=dil)
        X_extra = np.array(np.where(self.im_segm_bin_dil>self.im_segm_bin)).T
        X = np.array(np.where(self.im_segm_bin)).T
        from scipy.spatial import KDTree
        Xb = X[KDTree(X).query(X_extra)[1]]
        #self.im_segm_ = self.im_segm.copy()
        self.im_segm[tuple(X_extra.T)] = self.im_segm[tuple(Xb.T)] 
        self.shapesm = self.im_segm.shape
        cells = np.unique(self.im_segm)
        cms = nd.center_of_mass(self.im_segm>-1,self.im_segm,cells)
        self.cms = {c_+self.ifov*10**6:(cm_*self.shape/self.shapesm).astype(int) for c_,cm_ in zip(cells,cms) if c_>0}
def get_counts_per_cell(self,th_cor=0.5):
    dic_th = self.dic_th
    icol = self.icol
    im_segm = self.im_segm
    shapesm = self.im_segm.shape
    shape = self.shape

    Xh = self.Xh.copy()
    cor = Xh[:,-2]
    h = Xh[:,-1]
    keep = h>dic_th.get(icol,0)
    Xh = Xh[keep]
    txyz = np.array(self.dic_drift['txyz'])
    txyz[txyz>0]=0
    Xcms = Xh[:,:3]-txyz
    Xred = np.round((Xcms/shape)*shapesm).astype(int)
    good = ~np.any((Xred>=shapesm)|(Xred<0),axis=-1)
    Xh,Xred = Xh[good],Xred[good]
    self.Xred = Xred
    icells = im_segm[tuple(Xred.T)]
    cells,cts = np.unique(icells[(Xh[:,-2]>th_cor)],return_counts=True)
    self.good_counts = {c_+self.ifov*10**6:ct_ for c_,ct_ in zip(cells,cts) if c_>0}
    cells,cts = np.unique(icells[(Xh[:,-2]<th_cor)],return_counts=True)
    self.bad_counts = {c_+self.ifov*10**6:ct_ for c_,ct_ in zip(cells,cts) if c_>0}
def get_int_im1_im2(im1,im2,th_int=0.5):
    inters = ((im1>0)&(im2>0)).astype(int)
    im1_in2 = im1*inters
    N1max = np.max(im1)+1
    im2_in1 = im2*inters*N1max
    iint,cts = np.unique(im1_in2+im2_in1,return_counts=True)
    c1,cts1 = np.unique(im1,return_counts=True)
    dic_c1 = {c_:ct_ for c_,ct_ in zip(c1,cts1) if c_>0}
    c2,cts2 = np.unique(im2,return_counts=True)
    dic_c2 = {c_:ct_ for c_,ct_ in zip(c2,cts2) if c_>0}
    dic_int= {(c1,c2):(ct/dic_c1[c1],ct/dic_c2[c2]) for c2,c1,ct in zip(iint//N1max,iint%N1max,cts) 
         if (c1>0) and (c2>0) and (c1!=c2)}
    objs1 = nd.find_objects(im1)
    objs2 = nd.find_objects(im2)
    for cch in dic_int:
        c1,c2 = cch
        ic1,ic2 = dic_int[cch]
        obj1,obj2 = objs1[c1-1],objs2[c2-1]
        if (ic1>th_int) or (ic2>th_int):
            c_ = np.min([c1,c2])
            im1[obj1][im1[obj1]==c1]=c_
            im2[obj2][im2[obj2]==c2]=c_
    return im1,im2
def stitch3D(im_segm,niter=5,th_int=0.5):
    for it_ in range(niter):
        for iim in range(len(im_segm)-1):
            im_segm[iim],im_segm[iim+1] = get_int_im1_im2(im_segm[iim],im_segm[iim+1],th_int=th_int)
    return im_segm
from scipy import ndimage as nd

In [68]:
pa = post_analysis(r'Y:\Glass_MERFISH\CGBB_ChrX_01_31_2023_Analysis\*_C*_Xh_RNAs.pkl',force=True)

In [80]:
pa.fl = fl
pa.set_ = '_C'
pa.ifov = get_ifov(fl)

In [81]:
load_segm(pa,force=True,vol_th = 1000,dil=5,stitch_th=0.5)

In [82]:
pa.shape,pa

array([  60, 2900, 2900])

In [52]:
abeta = r"Y:\Glass_MERFISH\CGBB_ChrX_01_31_2023\H79_4G8allPos_AD"
abeta_bk = r"Y:\Glass_MERFISH\CGBB_ChrX_01_31_2023\H08_cleanAb2_AD"
seg_fld = r"Y:\Glass_MERFISH\CGBB_ChrX_01_31_2023\H0_AD"

In [53]:
fls = np.sort(glob.glob(abeta+os.sep+'*.zarr'))

In [56]:
fl =  fls[69]
fl_bk = fl.replace(abeta,abeta_bk)
fl_seg = fl.replace(abeta,seg_fld)
im = read_im(fl)
im_bk = read_im(fl_bk)
im_seg = read_im(fl_seg)
ncols,sz,sx,sy = im.shape
tzxy,_ = get_tzxy_fast(im,im_seg,sz=10)
print(tzxy,_)
tzxy_bk,_ = get_tzxy_fast(im_bk,im,sz=10)
print(tzxy_bk,_)

[  0 -74  27] [array([  0, -73,  27], dtype=int64), array([  0, -74,  27], dtype=int64), array([  0, -74,  27], dtype=int64), array([  0, -74,  28], dtype=int64), array([  0, -75,  27], dtype=int64)]
[  0 -14   5] [array([  0, -13,   5], dtype=int64), array([  0, -14,   4], dtype=int64), array([  0, -14,   5], dtype=int64), array([  0, -14,   4], dtype=int64), array([  0, -14,   5], dtype=int64)]


In [62]:
im_dif = im.astype(np.float32)-da.roll(im_bk.astype(np.float32),tzxy_bk,axis=[1,2,3])
im_diff = da.roll(im_dif,tzxy,axis=[1,2,3])

In [63]:
tzxy

array([  0, -74,  27])

In [60]:
from dask import array as da
import napari
viewer = napari.view_image(da.roll(im,tzxy,axis=[1,2,3]),name='Abeta')
viewer.add_image(da.roll(im_bk,tzxy+tzxy_bk,axis=[1,2,3]),name='Abeta_BK')
viewer.add_image(da.roll(im_dif,tzxy,axis=[1,2,3]),name='Abeta_diff')
viewer.add_image(im_seg)

<Image layer 'im_seg' at 0x18ca9aa4ee0>

In [17]:
im_bk[-1]

dask.array<getitem, shape=(90, 2900, 2900), dtype=uint16, chunksize=(1, 2900, 2900), chunktype=numpy.ndarray>

In [ ]:
Y:\Glass_MERFISH\CGBB_ChrX_01_31_2023_Analysis\Segmentation